In [371]:
import numpy as np
import pandas as pd
import janitor
import warnings
warnings.filterwarnings('ignore')
from datetime import datetime
import glob

In [372]:
path = "/Users/lee14257/Development/CMU/Perspectives_in Data_Science/Project/2019"
all_files = glob.glob(path + "/*.csv")

li = []

for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    li.append(df)

bike2019 = pd.concat(li, axis=0, ignore_index=True)

In [373]:
len(bike2019)

3398417

In [374]:
bike2019.head()

,Duration,Start date,End date,Start station number,Start station,End station number,End station,Bike number,Member type
0,429,2019-10-01 00:01:59,2019-10-01 00:09:08,31214,17th & Corcoran St NW,31203,14th & Rhode Island Ave NW,W23731,Member
1,1935,2019-10-01 00:03:07,2019-10-01 00:35:23,31269,3rd St & Pennsylvania Ave SE,31269,3rd St & Pennsylvania Ave SE,W22377,Member
2,563,2019-10-01 00:03:51,2019-10-01 00:13:14,31214,17th & Corcoran St NW,31251,12th & L St NW,W23473,Member
3,625,2019-10-01 00:05:34,2019-10-01 00:15:59,31301,Ward Circle / American University,31317,Wisconsin Ave & Ingomar St NW,W21371,Member
4,312,2019-10-01 00:05:47,2019-10-01 00:10:59,31519,1st & O St NW,31109,7th & T St NW,W23666,Member


In [375]:
bike2019['Start date'] = pd.to_datetime(bike2019['Start date'], utc=False)
bike2019['End date'] = pd.to_datetime(bike2019['End date'], utc=False)

In [376]:
# Number of unique stations
len(bike2019['End station number'].unique())

581

## Create Availability table

In [377]:
# Create Availability table
drop_df = bike2019[['Start station number', 'Start station', 'Start date', 'Bike number']].assign(act=-1)
drop_df.columns = ['station_id', 'station_name', 'date', 'bike_id', 'act']
pick_df = bike2019[['End station number', 'End station', 'End date', 'Bike number']].assign(act=1)
pick_df.columns = ['station_id', 'station_name', 'date', 'bike_id', 'act']
act_df = pd.concat([drop_df, pick_df])
act_df = act_df.assign(reshuffle=0)

In [378]:
act_df = act_df.sort_values(['station_id', 'date'])

In [379]:
act_df

,station_id,station_name,date,bike_id,act,reshuffle
2741430,0,22nd & H St NW,2019-12-01 16:05:41,W23121,-1,0
2741870,0,22nd & H St NW,2019-12-01 19:06:05,W24663,1,0
2741962,0,22nd & H St NW,2019-12-01 19:44:08,W24663,-1,0
2742027,0,22nd & H St NW,2019-12-01 20:14:23,W24118,-1,0
2742043,0,22nd & H St NW,2019-12-01 20:30:34,W21085,-1,0
...,...,...,...,...,...,...
2886440,32609,W Columbia St & N Washington St,2019-12-28 14:16:09,W00632,1,0
2886446,32609,W Columbia St & N Washington St,2019-12-28 14:16:24,W00420,1,0
2886696,32609,W Columbia St & N Washington St,2019-12-28 14:17:07,W00632,-1,0
2886697,32609,W Columbia St & N Washington St,2019-12-28 14:17:08,W00420,-1,0


In [380]:
#act_df.to_csv("transformed_data_201909.csv")

## Identify Reshuffled rows

In [381]:
# Add van reshuffled as additional rows
df_reshuffle = act_df.sort_values(['bike_id', 'date'])
df_reshuffle.head()

,station_id,station_name,date,bike_id,act,reshuffle
1363967,31124,14th & Irving St NW,2019-11-21 08:47:33,21054,-1,0
1363967,31101,14th & V St NW,2019-11-21 08:54:07,21054,1,0
1364146,31101,14th & V St NW,2019-11-21 08:55:49,21054,-1,0
1364146,31214,17th & Corcoran St NW,2019-11-21 09:02:29,21054,1,0
1364411,31214,17th & Corcoran St NW,2019-11-21 09:12:15,21054,-1,0


In [382]:
# Shift previous stations to compare and determine reshuffled rows
df_reshuffle['station_id_prev'] = df_reshuffle['station_id'].shift()
df_reshuffle['station_name_prev'] = df_reshuffle['station_name'].shift()
df_reshuffle['bike_id_prev'] = df_reshuffle['bike_id'].shift()
df_reshuffle['date_prev'] = df_reshuffle['date'].shift()

In [383]:
# Rows that need reshuffle (add rows)
need_reshuffle = df_reshuffle[(df_reshuffle['act'] == -1) & 
             (df_reshuffle['station_id'] != df_reshuffle['station_id_prev']) & 
             (df_reshuffle['bike_id'] == df_reshuffle['bike_id_prev'])]
need_reshuffle[['station_id_prev']] = need_reshuffle[['station_id_prev']].astype(int)
need_reshuffle.head()

,station_id,station_name,date,bike_id,act,reshuffle,station_id_prev,station_name_prev,bike_id_prev,date_prev
1365289,31124,14th & Irving St NW,2019-11-21 11:42:54,21054,-1,0,31227,13th St & New York Ave NW,21054,2019-11-21 09:21:34
1375451,31227,13th St & New York Ave NW,2019-11-22 16:42:55,21054,-1,0,31254,15th & K St NW,21054,2019-11-22 08:54:57
1388885,31302,Wisconsin Ave & Newark St NW,2019-11-25 07:55:12,21054,-1,0,31246,M St & Pennsylvania Ave NW,21054,2019-11-23 17:21:11
2761007,31299,Connecticut Ave & R St NW,2019-12-04 15:27:31,21054,-1,0,31253,19th & K St NW,21054,2019-12-03 08:12:44
2764396,31242,18th St & Pennsylvania Ave NW,2019-12-04 22:09:07,21054,-1,0,31203,14th & Rhode Island Ave NW,21054,2019-12-04 15:37:13


In [384]:
# Add mid point dates for reshuffling vans
need_reshuffle['date_1q'] = need_reshuffle['date_prev'] + ((need_reshuffle['date'] - need_reshuffle['date_prev'])//3)
need_reshuffle['date_2q'] = need_reshuffle['date_prev'] + (2*(need_reshuffle['date'] - need_reshuffle['date_prev'])//3)
need_reshuffle['date_1q'] = need_reshuffle["date_1q"].dt.strftime("%Y-%m-%d %H:%M:%S")
need_reshuffle['date_2q'] = need_reshuffle["date_2q"].dt.strftime("%Y-%m-%d %H:%M:%S")
need_reshuffle.head()

,station_id,station_name,date,bike_id,act,reshuffle,station_id_prev,station_name_prev,bike_id_prev,date_prev,date_1q,date_2q
1365289,31124,14th & Irving St NW,2019-11-21 11:42:54,21054,-1,0,31227,13th St & New York Ave NW,21054,2019-11-21 09:21:34,2019-11-21 10:08:40,2019-11-21 10:55:47
1375451,31227,13th St & New York Ave NW,2019-11-22 16:42:55,21054,-1,0,31254,15th & K St NW,21054,2019-11-22 08:54:57,2019-11-22 11:30:56,2019-11-22 14:06:55
1388885,31302,Wisconsin Ave & Newark St NW,2019-11-25 07:55:12,21054,-1,0,31246,M St & Pennsylvania Ave NW,21054,2019-11-23 17:21:11,2019-11-24 06:12:31,2019-11-24 19:03:51
2761007,31299,Connecticut Ave & R St NW,2019-12-04 15:27:31,21054,-1,0,31253,19th & K St NW,21054,2019-12-03 08:12:44,2019-12-03 18:37:39,2019-12-04 05:02:35
2764396,31242,18th St & Pennsylvania Ave NW,2019-12-04 22:09:07,21054,-1,0,31203,14th & Rhode Island Ave NW,21054,2019-12-04 15:37:13,2019-12-04 17:47:51,2019-12-04 19:58:29


In [385]:
# Create Reshuffle rows
reshuffle_pick = need_reshuffle[['station_id_prev', 'station_name_prev', 'date_1q', 'bike_id_prev']].assign(act=-1)
reshuffle_pick.columns = ['station_id', 'station_name', 'date', 'bike_id', 'act']
reshuffle_drop = need_reshuffle[['station_id', 'station_name', 'date_2q', 'bike_id']].assign(act=1)
reshuffle_drop.columns = ['station_id', 'station_name', 'date', 'bike_id', 'act']
act_df_reshuffle = pd.concat([reshuffle_pick, reshuffle_drop])
act_df_reshuffle = act_df_reshuffle.assign(reshuffle=1)

In [386]:
act_df_reshuffle.sort_values(['bike_id', 'date'])

,station_id,station_name,date,bike_id,act,reshuffle
1365289,31227,13th St & New York Ave NW,2019-11-21 10:08:40,21054,-1,1
1365289,31124,14th & Irving St NW,2019-11-21 10:55:47,21054,1,1
1375451,31254,15th & K St NW,2019-11-22 11:30:56,21054,-1,1
1375451,31227,13th St & New York Ave NW,2019-11-22 14:06:55,21054,1,1
1388885,31246,M St & Pennsylvania Ave NW,2019-11-24 06:12:31,21054,-1,1
...,...,...,...,...,...,...
2898011,31622,13th & D St NE,2019-12-30 04:13:50,W24878,1,1
1828882,31500,4th & W St NE,2019-08-02 18:24:16,W99910,-1,1
1828882,31062,Roosevelt Island,2019-08-04 01:18:36,W99910,1,1
103786,31223,Convention Center / 7th & M St NW,2019-08-27 04:24:42,W99910,-1,1


## Add original and reshuffle table together

In [387]:
# Add original and reshuffled table
act_df = act_df.reset_index(drop=True)
act_df_reshuffle = act_df_reshuffle.reset_index(drop=True)
final_act_df = pd.concat([act_df, act_df_reshuffle], ignore_index=True)
final_act_df.head()

,station_id,station_name,date,bike_id,act,reshuffle
0,0,22nd & H St NW,2019-12-01 16:05:41,W23121,-1,0
1,0,22nd & H St NW,2019-12-01 19:06:05,W24663,1,0
2,0,22nd & H St NW,2019-12-01 19:44:08,W24663,-1,0
3,0,22nd & H St NW,2019-12-01 20:14:23,W24118,-1,0
4,0,22nd & H St NW,2019-12-01 20:30:34,W21085,-1,0


In [388]:
final_act_df = final_act_df.sort_values(['station_id', 'date'])
len(final_act_df)

7494226

In [389]:
# Add availability column through cumsum
final_act_df = final_act_df.assign(availability = final_act_df.groupby('station_id')['act'].transform(pd.Series.cumsum))

In [390]:
final_act_df

,station_id,station_name,date,bike_id,act,reshuffle,availability
7292191,0,22nd & H St NW,2019-11-30 08:47:13,W21085,1,1,1
7410914,0,22nd & H St NW,2019-11-30 16:15:12,W23121,1,1,2
7167468,0,22nd & H St NW,2019-11-30 19:00:23,W00376,1,1,3
7201773,0,22nd & H St NW,2019-12-01 08:36:20,W00999,1,1,4
7481543,0,22nd & H St NW,2019-12-01 13:28:46,W24118,1,1,5
...,...,...,...,...,...,...,...
6796829,32609,W Columbia St & N Washington St,2019-12-28 14:16:09,W00632,1,0,-9
6796830,32609,W Columbia St & N Washington St,2019-12-28 14:16:24,W00420,1,0,-8
6796831,32609,W Columbia St & N Washington St,2019-12-28 14:17:07,W00632,-1,0,-9
6796832,32609,W Columbia St & N Washington St,2019-12-28 14:17:08,W00420,-1,0,-10


In [391]:
#act_df.to_csv("transformed_data_201909.csv")

## Add bike capacity column

In [392]:
capacity = pd.read_csv("/Users/lee14257/Development/CMU/Perspectives_in Data_Science/Project/bikeshare_capacity.csv")

In [393]:
capacity = capacity[['NAME', 'CAPACITY']]
capacity.columns = ['station_name', 'tot_capacity']

In [394]:
capacity

,station_name,tot_capacity
0,John McCormack Dr & Michigan Ave NE,11
1,5th & K St NW,19
2,19th & East Capitol St SE,15
3,Park Rd & Holmead Pl NW,19
4,Good Hope & Naylor Rd SE,19
...,...,...
652,Kingman Island/The Fields at RFK,19
653,Kenilworth Terrace & Hayes St. NE,19
654,National Harbor Carousel,15
655,Valley Ave & Wheeler Rd SE,19


In [395]:
final_act_df = pd.merge(final_act_df,capacity,on='station_name',how='left')

In [396]:
final_act_df

,station_id,station_name,date,bike_id,act,reshuffle,availability,tot_capacity
0,0,22nd & H St NW,2019-11-30 08:47:13,W21085,1,1,1,17.0
1,0,22nd & H St NW,2019-11-30 16:15:12,W23121,1,1,2,17.0
2,0,22nd & H St NW,2019-11-30 19:00:23,W00376,1,1,3,17.0
3,0,22nd & H St NW,2019-12-01 08:36:20,W00999,1,1,4,17.0
4,0,22nd & H St NW,2019-12-01 13:28:46,W24118,1,1,5,17.0
...,...,...,...,...,...,...,...,...
7494221,32609,W Columbia St & N Washington St,2019-12-28 14:16:09,W00632,1,0,-9,12.0
7494222,32609,W Columbia St & N Washington St,2019-12-28 14:16:24,W00420,1,0,-8,12.0
7494223,32609,W Columbia St & N Washington St,2019-12-28 14:17:07,W00632,-1,0,-9,12.0
7494224,32609,W Columbia St & N Washington St,2019-12-28 14:17:08,W00420,-1,0,-10,12.0


In [397]:
final_act_df.to_csv("bike2019_transformed.csv")

In [398]:
final_act_df[final_act_df['station_id'] == 0]

,station_id,station_name,date,bike_id,act,reshuffle,availability,tot_capacity
0,0,22nd & H St NW,2019-11-30 08:47:13,W21085,1,1,1,17.0
1,0,22nd & H St NW,2019-11-30 16:15:12,W23121,1,1,2,17.0
2,0,22nd & H St NW,2019-11-30 19:00:23,W00376,1,1,3,17.0
3,0,22nd & H St NW,2019-12-01 08:36:20,W00999,1,1,4,17.0
4,0,22nd & H St NW,2019-12-01 13:28:46,W24118,1,1,5,17.0
...,...,...,...,...,...,...,...,...
1023,0,22nd & H St NW,2019-12-31 18:34:11,W24253,-1,0,2,17.0
1024,0,22nd & H St NW,2019-12-31 20:56:48,W20562,-1,0,1,17.0
1025,0,22nd & H St NW,2019-12-31 21:06:09,W24344,-1,0,0,17.0
1026,0,22nd & H St NW,2019-12-31 22:55:53,W24235,1,0,1,17.0
